In [30]:
import numpy as np
import pandas as pd
import xarray as xr
from pycontrails import Flight, Aircraft
from pycontrails.datalib.ecmwf import ERA5
from pycontrails.models.emissions import Emissions
from pycontrails.core.met_var import (
    AirTemperature,
    SpecificHumidity,
)

In [31]:
start_date = "2022-03-01 00:00:00"
end_date = "2022-03-01 23:00:00"

pressure_levels = [300, 250, 200]

In [32]:
era5pl = ERA5(
    time=(start_date, end_date),
    variables=(AirTemperature, SpecificHumidity),
    pressure_levels=pressure_levels,
)

In [33]:
# download data from ERA5 (or open from cache)
met = era5pl.open_metdataset()

In [34]:
# demo synthetic flight
flight_attrs = {
    "flight_id": "test",
    # set constants along flight path
    #"true_airspeed": 226.099920796651,  # true airspeed, m/s
    "thrust": 0.22,  # thrust_setting
    "nvpm_ei_n": 1.897462e15,  # non-volatile emissions index
    "aircraft_type": "E190",
    "wingspan": 48,  # m
    "n_engine": 2,
}

In [35]:
# Example flight
df = pd.DataFrame()
df["longitude"] = np.linspace(-25, -40, 100)
df["latitude"] = np.linspace(34, 40, 100)
df["altitude"] = np.linspace(10900, 10900, 100)
df["engine_efficiency"] = np.linspace(0.34, 0.35, 100)
df["fuel_flow"] = np.linspace(2.1, 2.4, 100)  # kg/s
df["aircraft_mass"] = np.linspace(154445, 154345, 100)  # kg
df["time"] = pd.date_range("2022-03-01T00:15:00", "2022-03-01T02:30:00", periods=100)
df["true_airspeed"] = np.linspace(230, 230, 100)  # m/s

In [36]:
flight = Flight(df, attrs=flight_attrs)

In [37]:
flight

Flight [8 keys x 100 length, 7 attributes]
	Keys: longitude, latitude, altitude, engine_efficiency, fuel_flow, ..., time
	Attributes:
	time                [2022-03-01 00:15:00, 2022-03-01 02:30:00]
	longitude           [-40.0, -25.0]
	latitude            [34.0, 40.0]
	altitude            [10900.0, 10900.0]
	flight_id           test
	thrust              0.22
	nvpm_ei_n           1897462000000000.0
	aircraft_type       E190
	wingspan            48
	n_engine            2
	crs                 EPSG:4326

In [38]:
# Intersect with met data
flight["AirTemperature"] = flight.intersect_met(met["air_temperature"], method="nearest")

In [39]:
flight["SpecificHumidity"] = flight.intersect_met(met["specific_humidity"], method="nearest")

In [40]:
flight.attrs["engine_uid"] = "CFM56-2-C5"


In [41]:
flight

Flight [10 keys x 100 length, 8 attributes]
	Keys: longitude, latitude, altitude, engine_efficiency, fuel_flow, ..., SpecificHumidity
	Attributes:
	time                [2022-03-01 00:15:00, 2022-03-01 02:30:00]
	longitude           [-40.0, -25.0]
	latitude            [34.0, 40.0]
	altitude            [10900.0, 10900.0]
	flight_id           test
	thrust              0.22
	nvpm_ei_n           1897462000000000.0
	aircraft_type       E190
	wingspan            48
	n_engine            2
	crs                 EPSG:4326
	engine_uid          CFM56-2-C5

In [42]:
emi = Emissions(met=met)

/home/ktait98/miniconda3/envs/traffic/lib/python3.10/site-packages/pycontrails/core/models.py:189: UserWarning: Met data appears to have originated from ECMWF and no humidity scaling is enabled. For ECMWF data, consider using one of 'ConstantHumidityScaling', 'ExponentialBoostHumidityScaling', or 'ExponentialBoostLatitudeCorrectionHumidityScaling'. For example: 
>>> from pycontrails.models import ConstantHumidityScaling
>>> Emissions(met=met, ..., humidity_scaling=ConstantHumidityScaling(rhi_adj=0.99))
  warnings.warn(


In [43]:
flight_emi = emi.eval(source=flight)

/home/ktait98/miniconda3/envs/traffic/lib/python3.10/site-packages/pycontrails/models/emissions/emissions.py:346: UserWarning: Cannot find 'engine_uid' CFM56-2-C5 in EDB. A constant emissions will be used.
  warnings.warn(


In [44]:
flight_emi

Flight [30 keys x 100 length, 20 attributes]
	Keys: longitude, latitude, altitude, engine_efficiency, fuel_flow, ..., nvpm_number
	Attributes:
	time                [2022-03-01 00:15:00, 2022-03-01 02:30:00]
	longitude           [-40.0, -25.0]
	latitude            [34.0, 40.0]
	altitude            [10900.0, 10900.0]
	flight_id           test
	thrust              0.22
	nvpm_ei_n           1897462000000000.0
	aircraft_type       E190
	wingspan            48
	n_engine            2
	crs                 EPSG:4326
	engine_uid          CFM56-2-C5
	gaseous_data_source Constant
	nvpm_data_source    Constant
	total_co2           57534.00545454588
	total_h2o           22401.654545454712
	total_so2           21.855272727272887
	total_sulphates     0.44602597402597727
	total_oc            0.36425454545454816
	total_nox           275.74069090909296
	total_co            65.74794545454594
	total_hc            9.470618181818255
	total_nvpm_mass     1.602720000000012
	total_nvpm_number   1.821272727272741e+19

In [48]:
from datetime import datetime

ds = xr.Dataset(
    {
        "random": (["latitude", "longitude", "level", "time"], np.ones((20, 15, 4, 5))),
        "ones": (["latitude", "longitude", "level", "time"], np.ones((20, 15, 4, 5))),
    },
    coords={
        "latitude": np.arange(-100, -80, 1.0),
        "longitude": np.arange(30, 45, 1.0),
        "level": np.arange(100, 500, 100),
        "time": [datetime(2021, 1, 1, h) for h in range(5)],
    },
)

In [49]:
ds

<xarray.Dataset>
Dimensions:    (latitude: 20, longitude: 15, level: 4, time: 5)
Coordinates:
  * latitude   (latitude) float64 -100.0 -99.0 -98.0 -97.0 ... -83.0 -82.0 -81.0
  * longitude  (longitude) float64 30.0 31.0 32.0 33.0 ... 41.0 42.0 43.0 44.0
  * level      (level) int64 100 200 300 400
  * time       (time) datetime64[ns] 2021-01-01 ... 2021-01-01T04:00:00
Data variables:
    random     (latitude, longitude, level, time) float64 1.0 1.0 ... 1.0 1.0
    ones       (latitude, longitude, level, time) float64 1.0 1.0 ... 1.0 1.0

In [51]:
from pycontrails import MetDataset

met = MetDataset(ds)

ValueError: Latitude contains values < -90 . Latitude values must be contained in the interval [-90, 90].